##SETUP

In [31]:
import os
import time
import timeit
import pandas as pd
import random
import numpy as np
# Android ADB
from scrcpy import Client, const
from subprocess import check_output,Popen
import threading

# Image processing
from PIL import Image
import cv2
import pytesseract
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm

import sys
# Start Scrcpy once per restart
if 'importlib' not in sys.modules:  
    start_scrcpy=True
    if start_scrcpy:
        proc = Popen(['C:\Programs\Scrcpy\scrcpy'], shell=True)
        time.sleep(6) # <-- sleep for 6''
        proc.terminate() # <-- terminate the process
# internal
import bot_core
import bot_perception

import importlib
importlib.reload(bot_core)
importlib.reload(bot_perception)


SLEEP_DELAY=0.1


bot = bot_core.Bot()

## RUN BOT

In [32]:
wait=0
combat = 0
grid_df =None
watch_ad = True
# Main loop
while(1):
    output = bot.battle_screen(True,True,floor=5,ad=watch_ad)
    if output[1]=='fighting':
        watch_ad = True
        wait = 0
        combat+=1
        if combat>50:
            bot.restart_RR()
            combat = 0
            continue
        battle_bar=trange(8)
        for i in battle_bar:
                time.sleep(0.2)
                # Spawn units
                bot.click(450,1360)
                # Upgrade units
                bot.mana_level([3,1,5,4],hero_power=True)
                # Try to merge units
                grid_df,unit_series,df_groups,info = bot.try_merge(prev_grid=grid_df)
                clear_output()
                display([f'{combat}, {i+1}/8 {output[1]}, {info}', 'Average age: '+ str(grid_df['Age'].mean().round(2)),grid_df,unit_series[0:5],df_groups])
    else:
        combat=0
        wait+=1
        if wait>40:
            print('RESTARTING')
            bot.restart_RR()
            watch_ad = False
            wait=0
        print(output[1],'wait count:',wait)

['8, 6/8 fighting, Full Grid - Merging!',
 'Average age: 10.13',
     grid_id         unit  probability  rank  rank_error position   Age
 0    icon_0   monkey.png        249.0     2       365.0   [0, 0]  17.0
 1    icon_1  crystal.png        230.0     2       205.0   [0, 1]   2.0
 2    icon_2   hunter.png        233.0     2       345.0   [0, 2]   8.0
 3    icon_3   zealot.png        340.0     1      7255.0   [0, 3]   0.0
 4    icon_4   zealot.png        361.0     2       255.0   [0, 4]   7.0
 5    icon_5  crystal.png        235.0     2       330.0   [1, 0]   4.0
 6    icon_6  crystal.png        203.0     2       345.0   [1, 1]  16.0
 7    icon_7  chemist.png        211.0     2       505.0   [1, 2]   5.0
 8    icon_8  crystal.png        194.0     2       345.0   [1, 3]   8.0
 9    icon_9   monkey.png        199.0     2       510.0   [1, 4]   9.0
 10  icon_10  chemist.png        198.0     2       370.0   [2, 0]   9.0
 11  icon_11   zealot.png        320.0     1      5605.5   [2, 1]   1.0

# Play Game

In [2]:
# Restart game
bot.restart_RR()

## Quick restart

In [ ]:
# Drop co-op game
for i in range(30):
    bot.restart_RR(True)